# Qcodes loading

In [1]:
# -*- coding: utf-8 -*-
"""
Created on January 27 2022

@author: FB
"""

import matplotlib.pyplot as plt
import numpy as np
import os
from time import sleep, monotonic

import qcodes as qc
from qcodes import Station, load_or_create_experiment, initialise_database, Measurement, load_by_run_spec, load_by_id
from qcodes.dataset.plotting import plot_dataset, plot_by_id
from qcodes.utils.dataset.doNd import do0d,do1d, do2d

qc.config.dataset.dond_plot = True

database_name = 'FF1B_2022_OPX' #database name file. Look at cell below for its location

In [2]:
dir_database = 'D:\database'
database_format = '.db'
database_path = os.path.join(dir_database, database_name + database_format)

qc.config.user.mainfolder = os.path.join(dir_database, database_name) #saves plots in png and pdf in the database folder
qc.logger.start_all_logging()
station = qc.Station()

qc.initialise_or_create_database_at(database_path) 
exp = load_or_create_experiment(
    experiment_name= 'T7_3',
    sample_name="FF1B"
)

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\T7_3\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\T7_3\.qcodes\logs\220906-28676-qcodes.log


In [3]:
STATION = qc.Station(config_file='config_20220506.yaml',use_monitor='True')

In [4]:
#qdac=STATION.load_instrument('qdac_B')
qdac_2=STATION.load_instrument('qdac_B_2')

2022-09-06 16:33:17,181 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 109 ¦ c:\Users\T7_3\miniconda3\envs\livestream\lib\site-packages\qcodes\station.py:568: QCoDeSDeprecationWarning: The use of a comma separated string for the limits keyword is deprecated. Use "an array like "[lower_lim, upper_lim]"" as an alternative.
  issue_deprecation_warning(



In [5]:
step_att=STATION.load_instrument('step_att_2')
print(step_att.attenuation())

Connected to: Mini-Circuits MN=RCDAT-6000-60 (serial:SN=11909040004, firmware:E5-2) in 0.06s
36.0


# Setting up buffered acquisition

In [6]:
from config_my_pulses_FB import config
from qm.qua import wait_for_trigger,reset_phase,program,update_frequency,for_,stream_processing,declare,declare_stream,wait,measure,play,save,fixed,demod,ramp,amp,if_,elif_,else_,align, ramp_to_zero
from qm.QuantumMachinesManager import QuantumMachinesManager

qmm=QuantumMachinesManager(host='192.168.15.128',port=80)
        #qmm.close_all_quantum_machines()
# qm=qmm.open_qm(config, close_other_machines=True)

2022-09-06 16:33:21,637 - qm - INFO - Performing health check
2022-09-06 16:33:21,649 - qm - INFO - Health check passed


In [19]:
def setup_buffered(steps=100,trigger = True):
    with program() as triggered_meas:
        I =declare(fixed)
        Q = declare(fixed)  
        I_stream = declare_stream()
        Q_stream = declare_stream()
        
        n = declare(int)
        if trigger:
            wait_for_trigger('bottom_right_DQD_readout')
        with for_(n,0,n<steps,n+1):
            measure('readout_pulse_3ms','bottom_right_DQD_readout',None,demod.full('cos', I), demod.full('sin', Q))
            save(I,I_stream)
            save(Q,Q_stream)

        with stream_processing():
            I_stream.buffer(steps).save('I')
            Q_stream.buffer(steps).save('Q')

    triggered_id = qm.compile(triggered_meas)
    return triggered_id

In [20]:
def start_triggered(triggered_id):
    p_job=qm.queue.add_compiled(triggered_id)
    return p_job.wait_for_execution()

def get_amp(job):
    I_handle = job.result_handles.get('I')
    Q_handle = job.result_handles.get('Q')
    Q_handle.wait_for_all_values()
    I_handle.wait_for_all_values()
    results_I = I_handle.fetch_all()
    results_Q = Q_handle.fetch_all()
    return np.sqrt(results_Q**2+results_I**2)


def get_phase(job):
    I_handle = job.result_handles.get('I')
    Q_handle = job.result_handles.get('Q')
    Q_handle.wait_for_all_values()
    I_handle.wait_for_all_values()
    results_I = I_handle.fetch_all()
    results_Q = Q_handle.fetch_all()
    return np.arctan2(results_Q,results_I)

from qcodes import Measurement
from time import sleep
import time
import re
# do2d_buffered takes as channel not (e.g.) qdac.ch01.v but qdac.ch01, as I need later (in ramp_voltages_2d_buffered) the channel number (obtained by int(re.findall("\d+", <string name>)[0]))
def do1d_buffered(chan, vstart, vend, steps ,step_length, job_id):    
    
    #dmm_lockin.sample.pretrigger_count(0)
    start = time.time()
    meas = Measurement()
    v1s = np.linspace(vstart, vend, steps)
    #v2s = np.linspace(fast_vstart, fast_vend, fast_steps)
    meas.register_parameter(chan.v)
    #meas.register_parameter(fast_chan.v)
    meas.register_custom_parameter('amp', 'amp', 'V', setpoints=(chan.v,))
    meas.register_custom_parameter('phase', 'phase', 'rad', setpoints=(chan.v,))
    ramp_time = step_length*steps
    with meas.run() as datasaver: 
        job = start_triggered(job_id)
        # sleep(2)
        
        meas_time = qdac_2.ramp_voltages( [int(re.findall("\d+", chan.short_name)[0])], [vstart], [vend], ramp_time)
                                                
        # sleep(meas_time)
        print(meas_time)
        amp_data = get_amp(job) #data = dmm.fetch()
        phase_data = get_phase(job)
        datasaver.add_result((chan.v, np.linspace(vstart,vend,steps)),
                            ('amp', amp_data.flatten()),
                            ('phase',phase_data.flatten()))    
    #dataset2D = datasaver.dataset
    end = time.time()
    # plt.pcolormesh(data)
    print(end - start)


## do1d buffered

In [10]:
# # %% 
# qdac_2.ch06.sync(1) #channel
# qdac_2.ch06.sync_delay(0)
# qdac_2.ch06.sync_duration(0.001)
# qdac_2.ch06.v.label = 'BNC 30'

# steps = 100 #number of pixel along slow axis
# time_per_pixel = 0.003 # in seconds
# triggered_id = setup_buffered(steps=steps)
# do1d_buffered(qdac_2.ch06, -0.2, -0.1 ,steps,time_per_pixel,triggered_id)
# qdac_2.BNC30(-0.150)

## 2D buffered

In [21]:
from qcodes import Measurement
from time import sleep
import time
import re
# do2d_buffered takes as channel not (e.g.) qdac.ch01.v but qdac.ch01, as I need later (in ramp_voltages_2d_buffered) the channel number (obtained by int(re.findall("\d+", <string name>)[0]))
def do2d_buffered(slow_chan, slow_vstart, slow_vend, slow_steps , fast_chan, fast_vstart, fast_vend, fast_steps ,step_length, job_id):    
    
    #dmm_lockin.sample.pretrigger_count(0)
    start = time.time()
    meas = Measurement()
    v1s = np.linspace(slow_vstart, slow_vend, slow_steps)
    v2s = np.linspace(fast_vstart, fast_vend, fast_steps)
    meas.register_parameter(slow_chan.v)
    meas.register_parameter(fast_chan.v)
    meas.register_custom_parameter('amp', 'amp', 'V', setpoints=(slow_chan.v,fast_chan.v,))
    meas.register_custom_parameter('phase', 'phase', 'rad', setpoints=(slow_chan.v,fast_chan.v,))
    
    with meas.run() as datasaver: 
        job = start_triggered(job_id)
        # sleep(2)
        meas_time = qdac_2.ramp_voltages_2d( slow_chans=[int(re.findall("\d+", slow_chan.short_name)[0])], slow_vstart=[slow_vstart], slow_vend=[slow_vend],
                                    fast_chans=[int(re.findall("\d+", fast_chan.short_name)[0])], fast_vstart=[fast_vstart], fast_vend=[fast_vend],
                                    step_length=step_length,slow_steps = slow_steps, fast_steps = fast_steps)
                                                
        # sleep(meas_time)
        print(meas_time)

        amp_data =  get_amp(job)  #data = dmm.fetch()
        phase_data = get_phase(job)
        datasaver.add_result((slow_chan.v, np.repeat(v1s,fast_steps)),
                            (fast_chan.v, np.tile(v2s,slow_steps)),
                            ('amp', amp_data.flatten()),
                            ('phase',phase_data.flatten()))    
    # #dataset2D = datasaver.dataset
    end = time.time()
    # # plt.pcolormesh(data)
    print(end - start)


In [22]:
do0d_job_id = setup_buffered(steps=1,trigger=False)

def do0d(BNC30_val, job_id = do0d_job_id):    
    
    #dmm_lockin.sample.pretrigger_count(0)
    start = time.time()
    meas = Measurement()
    # meas.register_parameter(slow_chan.v)
    meas.register_custom_parameter('BNC30','BNC30','V')
    meas.register_custom_parameter('amp', 'amp', 'V')
    meas.register_custom_parameter('phase', 'phase', 'rad')
    
    with meas.run() as datasaver: 
        job = start_triggered(job_id)
                                                
        amp_data =  get_amp(job)  #data = dmm.fetch()
        phase_data = get_phase(job)
        datasaver.add_result(('BNC30', BNC30_val),
                            ('amp', amp_data.flatten()[0]),
                            ('phase',phase_data.flatten()[0]))    
    
    return amp_data, phase_data

2022-09-06 16:37:54,230 - qm - INFO - Flags: 
2022-09-06 16:37:54,231 - qm - INFO - Compiling program


In [15]:
qdac_2.ch12.sync(1) #slow channel
qdac_2.ch12.sync_delay(0)
qdac_2.ch12.sync_duration(0.001)
qdac_2.ch12.v.label = 'BNC 36'
qdac_2.ch05.v.label = 'BNC 29'

In [11]:
qdac_2.BNC34()

-0.365005

In [16]:
qdac_2.BNC29()

-0.128694

In [17]:
qdac_2.BNC36()

-0.179095

In [14]:
qdac_2.BNC32()

-0.519996

In [15]:
qdac_2.BNC37()

-0.699991

In [19]:
qdac_2.BNC30()

-0.131991

In [22]:
qdac_2.BNC30(-0.132)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.128694)
qdac_2.BNC36(-0.179095)

Starting experimental run with id: 1151. 
2022-09-06 15:45:07,203 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.113529920578003


# Overhauser


In [8]:
# from config_my_pulses_FB import config
from qm.qua import wait_for_trigger,reset_phase,Cast,Math, switch_,case_,program,update_frequency,for_,stream_processing,declare,declare_stream,wait,measure,play,save,fixed,demod,ramp,amp,if_,elif_,else_,align, ramp_to_zero
from qm.QuantumMachinesManager import QuantumMachinesManager
#from qualang_tools.bakery.bakery import baking
from qm import SimulationConfig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.dpi']=200
from datetime import datetime


In [9]:
dividers = {'gate_36': 8.18*1e-3, 'gate_29': 8.27*1e-3}

## settings Overhauser

In [10]:
det_min_amp = -35 #mV negative in (1,1)
angle_degree = 130.51  #degrees, 0 degree aligned with BNC29 in positive volt.direction


In [11]:
initialize = {'gate_36': 0, #in mV
              'gate_29': 1}
measurement_point = {'gate_36': 0, #in mV
              'gate_29': 1}

det_min = {'gate_36': -det_min_amp*np.sin(angle_degree*np.pi/180), #in mV
           'gate_29': -det_min_amp*np.cos(angle_degree*np.pi/180)}

CW_amp = 0.25 #from OPX config file

duration_init_pulse_in_clk_cycles = 1000
duration_wait_after_init_in_clk_cycles = 1000
time_at_measurement_point_per_RF_burst_in_clk_cycles = 1375
total_time_at_measurement_point_in_clk_cycles = duration_wait_after_init_in_clk_cycles + 2*time_at_measurement_point_per_RF_burst_in_clk_cycles
corrD_duration_in_clk_cycles = 2000

#det_pulse_duration
v_duration_overhauser_pulse_np = np.arange(0,101,1) 
n_overhauser_pulses_steps = len(v_duration_overhauser_pulse_np)
#v_duration_overhauser_pulse_in_clk_cycles = v_duration_overhauser_pulse_np/4
#corrD, 5 us long
a7_np = -(initialize['gate_36']*duration_init_pulse_in_clk_cycles + det_min['gate_36']*v_duration_overhauser_pulse_np/4 + measurement_point['gate_36']*total_time_at_measurement_point_in_clk_cycles)/CW_amp*dividers['gate_36']/corrD_duration_in_clk_cycles
a8_np = -(initialize['gate_29']*duration_init_pulse_in_clk_cycles + det_min['gate_29']*v_duration_overhauser_pulse_np/4 + measurement_point['gate_29']*total_time_at_measurement_point_in_clk_cycles)/CW_amp*dividers['gate_29']/corrD_duration_in_clk_cycles

In [12]:
from qualang_tools.bakery.bakery import baking
CW_amp = 0.25 #from OPX config file

baked_waveforms = []
for overhauser_duration in v_duration_overhauser_pulse_np:
    with baking(config, padding_method='symmetric_l') as b:
        b.add_op('gate_36_pulse','gate_36',[CW_amp]*overhauser_duration+[0])
        b.add_op('gate_29_pulse','gate_29',[CW_amp]*overhauser_duration+[0])

        b.play('gate_36_pulse','gate_36')
        b.play('gate_29_pulse','gate_29')
    baked_waveforms.append(b)
    

## macro Overhauser

In [13]:
def run_overhauser_oscillations(): 
    amp_array = [('gate_36', (det_min['gate_36']/CW_amp*dividers['gate_36'])),
                 ('gate_29', (det_min['gate_29']/CW_amp*dividers['gate_29']))]
           
    with for_(v5,0,v5<n_overhauser_pulses_steps,v5+1):
        #initialize singlet
        #initialize singlet
        align("gate_36", "gate_29")
        play("CW"*amp((initialize['gate_36'])/CW_amp*dividers['gate_36']), "gate_36", duration=duration_init_pulse_in_clk_cycles)
        play("CW"*amp((initialize['gate_29'])/CW_amp*dividers['gate_29']), "gate_29", duration=duration_init_pulse_in_clk_cycles)
        #jumping  to reference point and waiting there
        align("gate_36", "gate_29")
        play("CW"*amp((-initialize['gate_36'] + measurement_point['gate_36'])/CW_amp*dividers['gate_36']), "gate_36", duration=duration_wait_after_init_in_clk_cycles)
        play("CW"*amp((-initialize['gate_29'] + measurement_point['gate_29'])/CW_amp*dividers['gate_29']), "gate_29", duration=duration_wait_after_init_in_clk_cycles)
        #measure reference signal
        align("gate_36", "gate_29", "bottom_right_DQD_readout")
        play("CW"*amp(0.0), "gate_36", duration=time_at_measurement_point_per_RF_burst_in_clk_cycles)
        play("CW"*amp(-0.0), "gate_29", duration=time_at_measurement_point_per_RF_burst_in_clk_cycles)
        measure("readout_pulse_3us", "bottom_right_DQD_readout", None, demod.full("cos", v1, "out1"), demod.full("sin", v2, "out1"))
        #save values
        save(v1, r1)
        save(v2, r2)
        #det_pulse
        align("gate_36", "gate_29")
        with switch_(v5):
            for j in range(n_overhauser_pulses_steps):
                with case_(j):
                    baked_waveforms[j].run(amp_array)
    
        #meas signal
        align("gate_36", "gate_29", "bottom_right_DQD_readout")
        play("CW"*amp(0.0), "gate_36", duration=time_at_measurement_point_per_RF_burst_in_clk_cycles)
        play("CW"*amp(0.0), "gate_29", duration=time_at_measurement_point_per_RF_burst_in_clk_cycles)
        wait(125) #to avoid resonator ringing at the measurement
        reset_phase("bottom_right_DQD_readout")  
        measure("readout_pulse_5us", "bottom_right_DQD_readout", None, demod.sliced("cos", a1, 19, "out1"), demod.sliced("sin", a2, 19, "out1"))
        # #dummy pulse for constant qubit cycle length
        # align("gate_36", "gate_29")
        # play("CW"*amp(0.0), "gate_36", duration = 4 + v_duration_overhauser_pulse_qua[n_overhauser_pulses_steps-1] - v_duration_overhauser_pulse_qua[v5])
        # play("CW"*amp(0.0), "gate_29", duration = 4 + v_duration_overhauser_pulse_qua[n_overhauser_pulses_steps-1] - v_duration_overhauser_pulse_qua[v5])
        
        #corrD of 5 us
        align("gate_36", "gate_29")
        play("CW"*amp(a7[v5]- measurement_point['gate_36']/CW_amp*dividers['gate_36']), "gate_36", duration=corrD_duration_in_clk_cycles)
        play("CW"*amp(a8[v5]- measurement_point['gate_29']/CW_amp*dividers['gate_29']), "gate_29", duration=corrD_duration_in_clk_cycles)
        align("gate_36", "gate_29")
        ramp_to_zero("gate_36", 1)
        ramp_to_zero("gate_29", 1)
        with for_(v3,0,v3<65,v3+1):                
            save(a1[v3], r3)
            save(a2[v3], r4)
        
        run_bayes_estimation()
        assign(estimated_field, Math.dot(v_fB_qua,Pf))
    return estimated_field

## Macro bayes

In [14]:
def run_bayes_estimation():
            #BEGIN BAYES
            #"integrate" for a chosen time bayes
            assign(integrated_value,Math.dot(a1,average_dot_vector)) #a1 is I_2 which we are integrating until integration_time_us

            assign(ind1,0)
            #threshould the integrated value to assign singlet / triplet
            assign(state[v5], Cast.to_int(integrated_value > threshold) ) #here be careful with threshold r_k should be 1 for S and -1 for T
            save(state[v5], state_str)
            assign(rk, Cast.to_fixed(state[v5]) - 0.5)

            #loop through deltaBz values and update probability distribution
            with for_(fB, fB_min, fB < fB_max+0.5*dfB, fB + dfB):
                assign(C, Math.cos2pi(fB*t_samples[v5]))
                assign(Pf[ind1], (0.5 + rk * (alpha + beta * C)) * Pf[ind1])
                assign(ind1, ind1 + 1)           
            #normalize
            assign(norm, 1 / Math.sum(Pf))
            with for_(ind1, 0, ind1 < Pf.length(), ind1 + 1):
                assign(Pf[ind1], Pf[ind1] * norm)
            

# macro phase gate

In [15]:
from qualang_tools.bakery.bakery import baking
CW_amp = 0.25 #from OPX config file
v_duration_phase_gate_pulse_np = np.arange(0,201)
n_phase_gate_pulses_tau_steps = len(v_duration_phase_gate_pulse_np)
baked_waveforms_phase_gate= []
for overhauser_duration in v_duration_phase_gate_pulse_np:
    with baking(config, padding_method='symmetric_l') as b_phase_gate:
        b_phase_gate.add_op('gate_36_pulse','gate_36',[CW_amp]*overhauser_duration+[0])
        b_phase_gate.add_op('gate_29_pulse','gate_29',[CW_amp]*overhauser_duration+[0])

        b_phase_gate.play('gate_36_pulse','gate_36')
        b_phase_gate.play('gate_29_pulse','gate_29')
    baked_waveforms_phase_gate.append(b_phase_gate)
    

In [16]:
a7_phase_gate_np = -(initialize['gate_36']*duration_init_pulse_in_clk_cycles + det_min['gate_36']*v_duration_phase_gate_pulse_np/4 + measurement_point['gate_36']*total_time_at_measurement_point_in_clk_cycles)/CW_amp*dividers['gate_36']/corrD_duration_in_clk_cycles
a8_phase_gate_np = -(initialize['gate_29']*duration_init_pulse_in_clk_cycles + det_min['gate_29']*v_duration_phase_gate_pulse_np/4 + measurement_point['gate_29']*total_time_at_measurement_point_in_clk_cycles)/CW_amp*dividers['gate_29']/corrD_duration_in_clk_cycles

In [17]:
def run_phase_gate(active_estimation): 
    amp_array = [('gate_36', (det_min['gate_36']/CW_amp*dividers['gate_36'])),
                 ('gate_29', (det_min['gate_29']/CW_amp*dividers['gate_29']))]
           
    with for_(v5,0,v5<n_phase_gate_pulses_steps,v5+1):
        #initialize singlet
        align("gate_36", "gate_29")
        play("CW"*amp((initialize['gate_36'])/CW_amp*dividers['gate_36']), "gate_36", duration=duration_init_pulse_in_clk_cycles)
        play("CW"*amp((initialize['gate_29'])/CW_amp*dividers['gate_29']), "gate_29", duration=duration_init_pulse_in_clk_cycles)
        #jumping  to reference point and waiting there
        align("gate_36", "gate_29")
        play("CW"*amp((-initialize['gate_36'] + measurement_point['gate_36'])/CW_amp*dividers['gate_36']), "gate_36", duration=duration_wait_after_init_in_clk_cycles)
        play("CW"*amp((-initialize['gate_29'] + measurement_point['gate_29'])/CW_amp*dividers['gate_29']), "gate_29", duration=duration_wait_after_init_in_clk_cycles)
        #measure reference signal
        align("gate_36", "gate_29", "bottom_right_DQD_readout")
        play("CW"*amp(0.0), "gate_36", duration=time_at_measurement_point_per_RF_burst_in_clk_cycles)
        play("CW"*amp(-0.0), "gate_29", duration=time_at_measurement_point_per_RF_burst_in_clk_cycles)
        measure("readout_pulse_3us", "bottom_right_DQD_readout", None, demod.full("cos", v1, "out1"), demod.full("sin", v2, "out1"))
        #save values
        with if_(active_estimation == 0):
            save(v1, r1_without_estimation)
            save(v2, r2_without_estimation)
            align("gate_36", "gate_29")
            with switch_(v_tau_dummy_phase_gate_in_ns_qua[v5]):
                for j in range(n_phase_gate_pulses_tau_steps):
                    with case_(j):
                        baked_waveforms_phase_gate[j].run(amp_array)
        
        with else_():
            save(v1, r1_with_estimation)
            save(v2, r2_with_estimation)
            with switch_(v_tau_phase_gate_in_ns[v5]):
                for j in range(n_phase_gate_pulses_tau_steps):
                    with case_(j):
                        baked_waveforms_phase_gate[j].run(amp_array) # to run tau associated to wanted phase rotation
        #det_pulse
        
        #meas signal
        align("gate_36", "gate_29", "bottom_right_DQD_readout")
        play("CW"*amp(0), "gate_36", duration=time_at_measurement_point_per_RF_burst_in_clk_cycles)
        play("CW"*amp(0), "gate_29", duration=time_at_measurement_point_per_RF_burst_in_clk_cycles)
        wait(125) #to avoid resonator ringing at the measurement
        reset_phase("bottom_right_DQD_readout")  
        measure("readout_pulse_3us", "bottom_right_DQD_readout", None, demod.full("cos", v1, "out1"), demod.full("sin", v2, "out1"))
        # #dummy pulse for constant qubit cycle length
        # align("gate_36", "gate_29")
        # play("CW"*amp(0.0), "gate_36", duration = 4 + v_duration_overhauser_pulse_qua[n_overhauser_pulses_steps-1] - v_duration_overhauser_pulse_qua[v5])
        # play("CW"*amp(0.0), "gate_29", duration = 4 + v_duration_overhauser_pulse_qua[n_overhauser_pulses_steps-1] - v_duration_overhauser_pulse_qua[v5])
        
        #corrD of 5 us
        align("gate_36", "gate_29")
        play("CW"*amp(a7_phase_gate_qua[v_tau_phase_gate_in_ns[v5]]- measurement_point['gate_36']/CW_amp*dividers['gate_36']), "gate_36", duration=corrD_duration_in_clk_cycles)
        play("CW"*amp(a8_phase_gate_qua[v_tau_phase_gate_in_ns[v5]]- measurement_point['gate_29']/CW_amp*dividers['gate_29']), "gate_29", duration=corrD_duration_in_clk_cycles)
        align("gate_36", "gate_29")
        ramp_to_zero("gate_36", 1)
        ramp_to_zero("gate_29", 1)
        with if_(active_estimation == 0):
            save(v1, r3_without_estimation)
            save(v2, r4_without_estimation)
        with else_():
            save(v1, r3_with_estimation)
            save(v2, r4_with_estimation)

# main

In [18]:
from qm.qua import *
n_repetitions = 1000
n_averages = 4
n_phase_gate_pulses_steps = 50
phase_min_unit_2_pi = 0
phase_max_unit_2_pi = 3.5#4*np.pi

v_phase_gate_unit_2pi_np = np.linspace(phase_min_unit_2_pi,phase_max_unit_2_pi,n_phase_gate_pulses_steps )

v_tau_dummy_phase_gate_in_ns_np = np.arange(0,n_phase_gate_pulses_steps,1)

readout_length = 4940
chunk_size = 76
slices = readout_length // chunk_size

#t_max = 100 #?
# defining bayesian estimation parameters
#note FB, and t_samp are changed in scale to account for fixed point range of OPX, they are multiplied together so the result should be unchanged.
threshold = 90e-6
fB_min = 0.5#originally 1e6 for 1MHz 
fB_max = 5# 60e6 for 60MHz
dfB = 0.1# 1e6 for 1MHz steps
v_fB = np.arange(fB_min,fB_max+0.5*dfB,dfB)
num_Bs = len(v_fB)
alpha = 0.25
beta = 0.5
t_samp = 1e-2 #normalization factor to avoid overflow?
integration_time_us = 3



n_chunks_to_integrate = int(np.floor(integration_time_us*1e3/chunk_size).item())
v_duration_overhauser_pulse_np_normalized = v_duration_overhauser_pulse_np*t_samp #for avoid overflow
with program() as prog:
    # SETTINGS OVERHAUSER
    v1 = declare(fixed, )
    v2 = declare(fixed, )
    a1 = declare(fixed, size=65) #where I_2 is saved, size is number of slices
    a2 = declare(fixed, size=65)
    v3 = declare(int, )
    v4 = declare(int, )
    v5 = declare(int, )
    v6 = declare(int, ) #averages for phase gate pulses
    v7 = declare(int, ) #for constructing array of tau-s for phase gate
    #differential measurement    
    # delta_I = declare(fixed)
    # delta_Q = declare(fixed)
    #vector of tau overhauser
    v_duration_overhauser_pulse_qua = declare(int, value =  v_duration_overhauser_pulse_np.tolist())
    
    a7 = declare(fixed, value = a7_np.tolist()) 
    a8 = declare(fixed, value = a8_np.tolist()) 
    
    a7_phase_gate_qua = declare(fixed, value = a7_phase_gate_np.tolist()) 
    a8_phase_gate_qua = declare(fixed, value = a8_phase_gate_np.tolist()) 
    
    r1 = declare_stream() #I_1 ref
    r2 = declare_stream() #Q_1 ref
    r3 = declare_stream() #I_2 sig
    r4 = declare_stream() #Q_2 sig

    #declaring all things for bayesian
    #alpha beta defined outside
    Pf = declare(fixed, value = (np.ones(num_Bs)/num_Bs).tolist())
    Pf_str = declare_stream()
    state = declare(int, size = n_overhauser_pulses_steps)
    state_str = declare_stream()
    t_samples = declare(fixed,value = v_duration_overhauser_pulse_np_normalized.tolist())

    average_dot_vector = declare(fixed,
                                value=[1/n_chunks_to_integrate]*n_chunks_to_integrate+[0]*(slices-n_chunks_to_integrate))
    integrated_value = declare(fixed)

    fB = declare(fixed)

    C = declare(fixed)
    ind1 = declare(int)
    rk = declare(fixed)
    norm = declare(fixed)
    ind2 = declare(int)
    ind3 = declare(int)
    #variables for conditional estimation
    active_estimation = declare(int) #if 0, run dummy phase gate pulse
     
    r1_with_estimation = declare_stream() #I_1 ref _with_estimation
    r2_with_estimation = declare_stream() #Q_1 ref _with_estimation   
    r3_with_estimation = declare_stream() #I_2 sig _with_estimation
    r4_with_estimation = declare_stream() #Q_2 sig _with_estimation
    
    r1_without_estimation = declare_stream() #I_1 ref
    r2_without_estimation = declare_stream() #Q_1 ref
    r3_without_estimation = declare_stream() #I_2 sig _without_estimation
    r4_without_estimation = declare_stream() #Q_2 sig _without_estimation

    # #variables phase pulse
    v_phase_gate_unit_2pi_qua = declare(fixed, value = v_phase_gate_unit_2pi_np.tolist())
    v_tau_phase_gate_in_ns = declare(int, size = n_phase_gate_pulses_steps)
    v_tau_dummy_phase_gate_in_ns_qua = declare(int,value = v_tau_dummy_phase_gate_in_ns_np.tolist())

    counter_phase_pulses = declare(int, value = 0)
    counter_phase_pulses_stream = declare_stream()

    index_baked_phase_gate = declare(int) # to pick the correct tau for phase gate pulse
    #bayesan estimation
    estimated_field = declare(fixed) #10^7 Hz unit
    counter_phase_pulses_stream = declare_stream()
    #to calculate mean field from distribution
    v_fB_qua = declare(fixed, value = v_fB.tolist())
    #debugging
    v_effective_tau_computed_stream = declare_stream()
    v_estimated_field_stream = declare_stream()
    with for_(v4,0,v4<n_repetitions,v4+1):
        estimated_field = run_overhauser_oscillations()
        
        #constructing array of taus
        with if_(estimated_field >=1):

            with for_(v7,0,v7<n_phase_gate_pulses_steps,v7+1):
                assign(v_tau_phase_gate_in_ns[v7],Cast.mul_int_by_fixed(100,(v_phase_gate_unit_2pi_qua[v7]/estimated_field)))
                
                # let's add conditions on tau, which MUST be between 16 ns (4 clk cycles) and 200 ns
                #with if_(v_tau_phase_gate_in_clk_cycles[v7] < 0):
                #    assign(v_tau_phase_gate_in_clk_cycles[v7],4)
                with if_(v_tau_phase_gate_in_ns[v7] > 200):
                    assign(v_tau_phase_gate_in_ns[v7],200)
                save(v_tau_phase_gate_in_ns[v7],v_effective_tau_computed_stream)

            with for_(v6,0,v6<n_averages,v6+1):
                assign(active_estimation,1)
                run_phase_gate(active_estimation)
            #random phase gate to show we are actually doing something that makes sense
            with for_(v6,0,v6<n_averages,v6+1):
                assign(active_estimation,0)
                run_phase_gate(active_estimation)

            assign(counter_phase_pulses,counter_phase_pulses+1)

        
        # with if_(estimated_field<=1.5):#pi/2 rotation possible for f<15.63MHz
        #     assign(time_pi_over_2_ns, 0.25/estimated_field) #ns, 1/(4*estimated_field)*100 reducing a divsion
        #     assign(time_pi_over_2_effective_clock_cycles,Cast.mul_int_by_fixed(25,time_pi_over_2_ns))
        
        # # with elif_(estimated_field > 1.5): #freq >16 MHz and < 39.06 MHz
        # with else_():
        #     assign(time_pi_over_2_ns, 1.25/estimated_field) #ns, 5/(4*estimated_field)*100 reducing a divsion
        #     assign(time_pi_over_2_effective_clock_cycles,Cast.mul_int_by_fixed(25,time_pi_over_2_ns))                   
    
        # run_exchange_oscillations(time_pi_over_2_effective_clock_cycles)    
        
        #save estimated field and pi/2 duration in terms of clk cycles
        save(estimated_field, v_estimated_field_stream)
        #save(time_pi_over_2_effective_clock_cycles, time_pi_over_2_effective_stream)
        #save and reset probabilities between repetitions
        with for_( ind3, 0 , ind3 < Pf.length(), ind3 + 1):
            save(Pf[ind3],Pf_str)
            assign(Pf[ind3],1/num_Bs)
    save(counter_phase_pulses,counter_phase_pulses_stream)

    with stream_processing():
        r1.save_all("I_1") #I_ref for estimating overhauser sequence
        r2.save_all("Q_1") #Q_ref for estimating overhauser sequence
        r3.buffer(n_repetitions,n_overhauser_pulses_steps,65).save_all("I_2") #I_sig for estimating overhauser sequence
        r4.buffer(n_repetitions,n_overhauser_pulses_steps,65).save_all("Q_2") #Q_sig for estimating overhauser sequence
        #saving state and probability for fB
        state_str.buffer(n_repetitions,n_overhauser_pulses_steps).save_all('state')
        Pf_str.buffer(n_repetitions,num_Bs).save_all('Pf_str')
        
        ### save phase gate results ###
        
        #with estimated phase
        r1_with_estimation.save_all("r1_with_estimation")
        r2_with_estimation.save_all("r2_with_estimation")
        r3_with_estimation.buffer(n_averages,n_phase_gate_pulses_steps).map(FUNCTIONS.average(0)).save_all("r3_with_estimation")
        r4_with_estimation.buffer(n_averages,n_phase_gate_pulses_steps).map(FUNCTIONS.average(0)).save_all("r4_with_estimation")
        
        #dummy phase gate
        r1_without_estimation.save_all("r1_without_estimation")
        r2_without_estimation.save_all("r2_without_estimation")
        r3_without_estimation.buffer(n_averages,n_phase_gate_pulses_steps).map(FUNCTIONS.average(0)).save_all("r3_without_estimation")
        r4_without_estimation.buffer(n_averages,n_phase_gate_pulses_steps).map(FUNCTIONS.average(0)).save_all("r4_without_estimation")
        counter_phase_pulses_stream.save_all('counter_phase_pulses')
        #estimated field
        v_estimated_field_stream.save_all("v_estimated_field")
        #debugging
        v_effective_tau_computed_stream.buffer(n_phase_gate_pulses_steps).save_all('v_effective_tau_computed')

qm = qmm.open_qm(config)
overhauser_id = qm.compile(prog)

2022-09-06 16:34:21,911 - qm - INFO - Flags: 
2022-09-06 16:34:21,912 - qm - INFO - Compiling program


In [29]:
p_job = qm.queue.add_compiled(overhauser_id)
job = p_job.wait_for_execution()
job.result_handles.wait_for_all_values()

all_data = {name : data.fetch_all(flat_struct=True) for name,data in job.result_handles._all_results.items()}


2022-09-06 16:30:37,440 - qm - INFO - Sending pre-compiled program to QOP


In [30]:
all_data.keys()

dict_keys(['I_1', 'Q_1', 'I_2', 'Q_2', 'state', 'Pf_str', 'r1_with_estimation', 'r2_with_estimation', 'r3_with_estimation', 'r4_with_estimation', 'r1_without_estimation', 'r2_without_estimation', 'r3_without_estimation', 'r4_without_estimation', 'counter_phase_pulses', 'v_estimated_field', 'v_effective_tau_computed'])

In [33]:
np.save('test_folder/all_data.npy',all_data)

# Full sweep

In [23]:
qdac_2.BNC30()

-0.131991

In [24]:
qdac_2.ch06.v()

-0.131991

In [39]:
start_times = []
stop_times = []
for bnc30_val in np.linspace(-0.13,-0.134,30):
    start_times.append(time.perf_counter())
    os.mkdir(f'data_folder/{bnc30_val}')

    qdac_2.ch06.v(bnc30_val)

    amp,phase = do0d(bnc30_val)
    np.save(f'data_folder/{bnc30_val}/sensor_data',np.array([bnc30_val,amp[0],phase[0]]))


    p_job = qm.queue.add_compiled(overhauser_id)
    job = p_job.wait_for_execution()
    job.result_handles.wait_for_all_values()
    all_data = {name : data.fetch_all(flat_struct=True) for name,data in job.result_handles._all_results.items()}

    np.save(f'data_folder/{bnc30_val}/all_data',all_data)


    stop_times.append(time.perf_counter())
    print(stop_times[-1]-start_times[-1])

qdac_2.BNC30(-0.132)


Starting experimental run with id: 1157. 
2022-09-06 16:48:18,888 - qm - INFO - Sending pre-compiled program to QOP
2022-09-06 16:48:19,246 - qm - INFO - Sending pre-compiled program to QOP
20.106133
Starting experimental run with id: 1158. 
2022-09-06 16:48:38,908 - qm - INFO - Sending pre-compiled program to QOP
2022-09-06 16:48:39,266 - qm - INFO - Sending pre-compiled program to QOP
17.332185900000013
Starting experimental run with id: 1159. 
2022-09-06 16:48:56,336 - qm - INFO - Sending pre-compiled program to QOP
2022-09-06 16:48:56,712 - qm - INFO - Sending pre-compiled program to QOP
18.160877000000028
Starting experimental run with id: 1160. 
2022-09-06 16:49:14,455 - qm - INFO - Sending pre-compiled program to QOP
2022-09-06 16:49:14,820 - qm - INFO - Sending pre-compiled program to QOP
20.324092899999982
Starting experimental run with id: 1161. 
2022-09-06 16:49:34,811 - qm - INFO - Sending pre-compiled program to QOP
2022-09-06 16:49:35,216 - qm - INFO - Sending pre-compile

In [36]:
test = np.load('data_folder/-0.134/all_data.npy',allow_pickle=True)[()]

In [38]:
test['I_2'].shape

(1000, 101, 65)